# Notebook 2: Preprocessing Data

In [23]:
# Importing required libraries and packages
import pandas as pd
import numpy as np
import geopandas as gpd
import warnings

from collections import defaultdict

warnings.filterwarnings("ignore")

In [2]:
# A function that combines one year's
# worth of data into a single dataframe
def mergecsv(directory,
             taxi_color,
             year):
    # Initialising the list of dataframes
    # with January's data
    dfs = []
    df = pd.read_csv(directory 
                     + "/" 
                     + taxi_color
                     + "_tripdata_" 
                     + str(year) 
                     + "-01.csv")
    dfs.append(df)
    # Adds February's to December's data
    # to the dataframe list
    for i in range(2,13):
        # Formats to two digits
        if i < 10:
            num = "0" + str(i)
        else:
            num = str(i)
            
        tempdf = pd.read_csv(directory 
                             + "/" 
                             + taxi_color
                             + "_tripdata_" 
                             + str(year) 
                             + "-" 
                             + num 
                             + ".csv")
        dfs.append(tempdf)
        
    final = pd.concat(dfs)
    return final

### TLC Data

In [3]:
# Initialise the directory with the TLC datasets
directory = "../raw_data"

# Obtain the entire 2019 and 2020 Green and Yellow 
# taxi data into their respective dataframes
yellow_2019 = mergecsv(directory, "yellow", 2019)
yellow_2020 = mergecsv(directory, "yellow", 2020)
green_2019 = mergecsv(directory, "green", 2019)
green_2020 = mergecsv(directory, "green", 2020)

In [4]:
print("Total Number of Instances: " + str((len(yellow_2019.index)) 
      + len(yellow_2020.index) 
      + len(green_2019.index) 
      + len(green_2020.index)))

Total Number of Instances: 116825619


In [5]:
# Removes all instances where:
# - there are no passengers,
# - there is no distance travelled,
# - payment is not by credit card
# - the fare amount is negative,
# - tip amount is negative

In [6]:
yellow_2019 = yellow_2019.loc[((yellow_2019['payment_type'] == 1)
                               & (yellow_2019['passenger_count'] > 0)
                               & (yellow_2019['trip_distance'] > 0)
                               & (yellow_2019['fare_amount'] >= 0)
                               & (yellow_2019['tip_amount'] >= 0))]

In [7]:
yellow_2020 = yellow_2020.loc[((yellow_2020['payment_type'] == 1)
                               & (yellow_2020['passenger_count'] > 0)
                               & (yellow_2020['trip_distance'] > 0)
                               & (yellow_2020['fare_amount'] >= 0)
                               & (yellow_2020['tip_amount'] >= 0))]

In [8]:
green_2019 = green_2019.loc[((green_2019['payment_type'] == 1)
                               & (green_2019['passenger_count'] > 0)
                               & (green_2019['trip_distance'] > 0)
                               & (green_2019['fare_amount'] >= 0)
                               & (green_2019['tip_amount'] >= 0))]

In [9]:
green_2020 = green_2020.loc[((green_2020['payment_type'] == 1)
                               & (green_2020['passenger_count'] > 0)
                               & (green_2020['trip_distance'] > 0)
                               & (green_2020['fare_amount'] >= 0)
                               & (green_2020['tip_amount'] >= 0))]

In [10]:
print("Total Number of Instances: " + str((len(yellow_2019.index)) 
      + len(yellow_2020.index) 
      + len(green_2019.index) 
      + len(green_2020.index)))

Total Number of Instances: 79809343


In [11]:
# Setting the index to start from zero
yellow_2019 = yellow_2019.reset_index(drop = True)
yellow_2020 = yellow_2020.reset_index(drop = True)
green_2019 = green_2019.reset_index(drop = True)
green_2020 = green_2020.reset_index(drop = True)

### Unemployment Data 

In [19]:
# Loads unemployment data
unemployment = pd.read_csv("../raw_data/unemployment.csv")
# Shows the dataframe
unemployment

,Borough,2019,2020
0,Bronx,5.3,16.0
1,Brooklyn,4.0,12.5
2,Manhattan,3.4,9.5
3,Queens,3.4,12.5
4,Staten Island,3.8,10.6


In [13]:
# Code adapted from Lab 2

# Loads the zone shape file
zone = pd.read_csv("../raw_data/taxi+_zone_lookup.csv")
sf = gpd.read_file("../raw_data/taxi_zones.shp")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# Obtain the unique boroughs for both Yellow
# and Green taxis
yellow_gdf = gpd.GeoDataFrame(pd.merge(yellow_2019,
                                       sf,
                                       left_on = 'PULocationID',
                                       right_on = 'LocationID')).drop('PULocationID',axis = 1)
green_gdf = gpd.GeoDataFrame(pd.merge(green_2019,
                                      sf,
                                      left_on = 'PULocationID',
                                      right_on = 'LocationID')).drop('PULocationID',axis = 1)

print(yellow_gdf['borough'].unique())
print(green_gdf['borough'].unique())

['Manhattan' 'Queens' 'Brooklyn' 'Bronx' 'Staten Island' 'EWR']
['Brooklyn' 'Queens' 'Manhattan' 'Bronx' 'Staten Island' 'EWR']


In [20]:
# Adding a row for EWR
new_row = {'Borough':'EWR',
           '2019': unemployment['2019'].mean(),
           '2020': unemployment['2020'].mean()}

unemployment = unemployment.append(new_row,
                                   ignore_index = True)

### Combining Data

In [21]:
boroughsID = defaultdict()
for borough in unemployment['Borough'].unique():
    boroughsID[borough] = []

boroughsID['Unknown'] = []
    
for row in zone.iterrows():
    boroughsID[row[1][1]].append(row[1][0])  
    

In [24]:
pu = yellow_2019['PULocationID'].to_numpy()
yellow_2019['borough'] = np.nan

for i in range(len(pu)):
    if pu[i] in boroughsID['Bronx']:
        borough = 'Bronx'
    elif pu[i] in boroughsID['Brooklyn']:
        borough = 'Brooklyn'
    elif pu[i] in boroughsID['Manhattan']:
        borough = 'Manhattan'
    elif pu[i] in boroughsID['Queens']:
        borough = 'Queens'
    elif pu[i] in boroughsID['Staten Island']:
        borough = 'Staten Island'
    elif pu[i] in boroughsID['EWR']:
        borough = 'EWR'
    else:
        borough = 'Unknown'
    yellow_2019['borough'][i] = borough
    
    
conditions = [
    (yellow_2019['borough'] == 'Bronx'),
    (yellow_2019['borough'] == 'Brooklyn'),
    (yellow_2019['borough'] == 'Manhattan'),
    (yellow_2019['borough'] == 'Queens'),
    (yellow_2019['borough'] == 'Staten Island'),
    (yellow_2019['borough'] == 'EWR'),
    (yellow_2019['borough'] == 'Unknown')]

choices = [unemployment['2019'][0],
           unemployment['2019'][1],
           unemployment['2019'][2],
           unemployment['2019'][3],
           unemployment['2019'][4],
           unemployment['2019'][5],
           0]

yellow_2019['unemployment'] = np.select(conditions, choices)

In [25]:
pu = yellow_2020['PULocationID'].to_numpy()
yellow_2020['borough'] = np.nan

for i in range(len(pu)):
    if pu[i] in boroughsID['Bronx']:
        borough = 'Bronx'
    elif pu[i] in boroughsID['Brooklyn']:
        borough = 'Brooklyn'
    elif pu[i] in boroughsID['Manhattan']:
        borough = 'Manhattan'
    elif pu[i] in boroughsID['Queens']:
        borough = 'Queens'
    elif pu[i] in boroughsID['Staten Island']:
        borough = 'Staten Island'
    elif pu[i] in boroughsID['EWR']:
        borough = 'EWR'
    else:
        borough = 'Unknown'
    yellow_2020['borough'][i] = borough
    
    
conditions = [
    (yellow_2020['borough'] == 'Bronx'),
    (yellow_2020['borough'] == 'Brooklyn'),
    (yellow_2020['borough'] == 'Manhattan'),
    (yellow_2020['borough'] == 'Queens'),
    (yellow_2020['borough'] == 'Staten Island'),
    (yellow_2020['borough'] == 'EWR'),
    (yellow_2020['borough'] == 'Unknown')]

choices = [unemployment['2020'][0],
           unemployment['2020'][1],
           unemployment['2020'][2],
           unemployment['2020'][3],
           unemployment['2020'][4],
           unemployment['2020'][5],
           0]

yellow_2020['unemployment'] = np.select(conditions, choices)

In [26]:
pu = green_2019['PULocationID'].to_numpy()
green_2019['borough'] = np.nan

for i in range(len(pu)):
    if pu[i] in boroughsID['Bronx']:
        borough = 'Bronx'
    elif pu[i] in boroughsID['Brooklyn']:
        borough = 'Brooklyn'
    elif pu[i] in boroughsID['Manhattan']:
        borough = 'Manhattan'
    elif pu[i] in boroughsID['Queens']:
        borough = 'Queens'
    elif pu[i] in boroughsID['Staten Island']:
        borough = 'Staten Island'
    elif pu[i] in boroughsID['EWR']:
        borough = 'EWR'
    else:
        borough = 'Unknown'
    green_2019['borough'][i] = borough
    
    
conditions = [
    (green_2019['borough'] == 'Bronx'),
    (green_2019['borough'] == 'Brooklyn'),
    (green_2019['borough'] == 'Manhattan'),
    (green_2019['borough'] == 'Queens'),
    (green_2019['borough'] == 'Staten Island'),
    (green_2019['borough'] == 'EWR'),
    (green_2019['borough'] == 'Unknown')]

choices = [unemployment['2019'][0],
           unemployment['2019'][1],
           unemployment['2019'][2],
           unemployment['2019'][3],
           unemployment['2019'][4],
           unemployment['2019'][5],
           0]

green_2019['unemployment'] = np.select(conditions, choices)

In [27]:
pu = green_2020['PULocationID'].to_numpy()
green_2020['borough'] = np.nan

for i in range(len(pu)):
    if pu[i] in boroughsID['Bronx']:
        borough = 'Bronx'
    elif pu[i] in boroughsID['Brooklyn']:
        borough = 'Brooklyn'
    elif pu[i] in boroughsID['Manhattan']:
        borough = 'Manhattan'
    elif pu[i] in boroughsID['Queens']:
        borough = 'Queens'
    elif pu[i] in boroughsID['Staten Island']:
        borough = 'Staten Island'
    elif pu[i] in boroughsID['EWR']:
        borough = 'EWR'
    else:
        borough = 'Unknown'
    green_2020['borough'][i] = borough
    
    
conditions = [
    (green_2020['borough'] == 'Bronx'),
    (green_2020['borough'] == 'Brooklyn'),
    (green_2020['borough'] == 'Manhattan'),
    (green_2020['borough'] == 'Queens'),
    (green_2020['borough'] == 'Staten Island'),
    (green_2020['borough'] == 'EWR'),
    (green_2020['borough'] == 'Unknown')]

choices = [unemployment['2020'][0],
           unemployment['2020'][1],
           unemployment['2020'][2],
           unemployment['2020'][3],
           unemployment['2020'][4],
           unemployment['2020'][5],
           0]

green_2020['unemployment'] = np.select(conditions, choices)

## Export to Preprocessing Folder 

In [28]:
# TLC Data
yellow_2019.to_csv('../preprocessed_data/yellow_2019.csv')
yellow_2020.to_csv('../preprocessed_data/yellow_2020.csv')
green_2019.to_csv('../preprocessed_data/green_2019.csv')
green_2020.to_csv('../preprocessed_data/green_2020.csv')
# Unemployment data
unemployment.to_csv('../preprocessed_data/unemployment.csv')